# Gold Price Forecasting with Geopolitical Risk
## Time-Series Forecasting using XGBoost

This notebook builds a forecasting model for next-day gold prices using:
- Historical gold and silver prices
- Geopolitical risk indices (GPRD, GPRD_ACT, GPRD_THREAT)
- Engineered features: lags, rolling statistics, calendar features

## 1. Imports & Path Setup

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import pickle

sns.set_style("whitegrid")

BASE_DIR = Path("..").resolve()
DATA_RAW_DIR = BASE_DIR / "data" / "raw"
OUTPUTS_PLOTS_DIR = BASE_DIR / "outputs" / "plots"
OUTPUTS_FORECASTS_DIR = BASE_DIR / "outputs" / "forecasts"
MODELS_DIR = BASE_DIR / "models"

OUTPUTS_PLOTS_DIR.mkdir(parents=True, exist_ok=True)
OUTPUTS_FORECASTS_DIR.mkdir(parents=True, exist_ok=True)
MODELS_DIR.mkdir(parents=True, exist_ok=True)

print(f"Base directory: {BASE_DIR}")
print(f"Data directory: {DATA_RAW_DIR}")

## 2. Load and Inspect Data

In [ ]:
file_path = DATA_RAW_DIR / "Gold-Silver-GeopoliticalRisk_HistoricalData.csv"
df = pd.read_csv(file_path)

# Normalize column names
df.columns = [c.strip().upper() for c in df.columns]

print(f"Dataset shape: {df.shape}")
df.head()

In [ ]:
df.info()

In [ ]:
# Convert date and clean data
df['DATE'] = pd.to_datetime(df['DATE'])
df = df.sort_values('DATE').set_index('DATE')

# Select relevant columns
df = df[['GOLD_PRICE', 'SILVER_PRICE', 'GPRD', 'GPRD_ACT', 'GPRD_THREAT']]

# Forward fill then backward fill missing values
df = df.ffill().bfill()

print(f"\nData after cleaning:")
print(f"Shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"\nMissing values:\n{df.isnull().sum()}")

In [ ]:
df.describe()

## 3. Exploratory Data Analysis

In [ ]:
# Gold & Silver time series
plt.figure(figsize=(12, 4))
plt.plot(df.index, df['GOLD_PRICE'], label='Gold', alpha=0.8)
plt.plot(df.index, df['SILVER_PRICE'], label='Silver', alpha=0.8)
plt.title("Gold & Silver Spot Prices (1985–2025)", fontsize=14)
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUTS_PLOTS_DIR / "gold_silver_timeseries.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Geopolitical Risk Index
plt.figure(figsize=(12, 3))
plt.plot(df.index, df['GPRD'], color='crimson', alpha=0.8)
plt.title("Geopolitical Risk Index (GPRD)", fontsize=14)
plt.xlabel('Date')
plt.ylabel('GPRD')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUTS_PLOTS_DIR / "gprd_timeseries.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Correlation matrix
plt.figure(figsize=(8, 6))
sns.heatmap(df.corr(), annot=True, cmap="coolwarm", center=0, fmt='.2f')
plt.title("Correlation Matrix", fontsize=14)
plt.tight_layout()
plt.savefig(OUTPUTS_PLOTS_DIR / "corr_matrix.png", dpi=300, bbox_inches='tight')
plt.show()

## 4. Feature Engineering

In [ ]:
# Create target: next-day GOLD_PRICE
df['GOLD_TARGET'] = df['GOLD_PRICE'].shift(-1)

print(f"Target variable created: GOLD_TARGET")
print(f"First few values: {df['GOLD_TARGET'].head()}")

In [ ]:
# Create lag features
lags = [1, 2, 5, 10, 20]

for lag in lags:
    df[f'GOLD_LAG_{lag}'] = df['GOLD_PRICE'].shift(lag)
    df[f'SILVER_LAG_{lag}'] = df['SILVER_PRICE'].shift(lag)
    df[f'GPRD_LAG_{lag}'] = df['GPRD'].shift(lag)

print(f"\nLag features created for lags: {lags}")
print(f"New columns: {[c for c in df.columns if 'LAG' in c][:6]}...")

In [ ]:
# Create rolling features
windows = [5, 10, 20]

for window in windows:
    df[f'GOLD_ROLL_MEAN_{window}'] = df['GOLD_PRICE'].rolling(window).mean()
    df[f'GOLD_ROLL_STD_{window}'] = df['GOLD_PRICE'].rolling(window).std()
    df[f'GPRD_ROLL_MEAN_{window}'] = df['GPRD'].rolling(window).mean()

print(f"\nRolling features created for windows: {windows}")
print(f"New columns: {[c for c in df.columns if 'ROLL' in c][:6]}...")

In [ ]:
# Create time-based features
df['YEAR'] = df.index.year
df['MONTH'] = df.index.month
df['DAYOFWEEK'] = df.index.dayofweek

print(f"\nTime-based features created: YEAR, MONTH, DAYOFWEEK")

In [ ]:
# Drop rows with NaN values
print(f"\nBefore dropping NaNs: {df.shape}")
df_model = df.dropna().copy()
print(f"After dropping NaNs: {df_model.shape}")
print(f"Rows dropped: {len(df) - len(df_model)}")

In [ ]:
df_model.info()

## 5. Train/Validation Split

In [ ]:
# Time-based split: train on data before 2020, validate on 2020+
cutoff_date = "2020-01-01"

train = df_model[df_model.index < cutoff_date]
val = df_model[df_model.index >= cutoff_date]

feature_cols = [c for c in df_model.columns if c != 'GOLD_TARGET']
X_train = train[feature_cols]
y_train = train['GOLD_TARGET']
X_val = val[feature_cols]
y_val = val['GOLD_TARGET']

print(f"Train set: {X_train.shape}, period: {train.index.min()} to {train.index.max()}")
print(f"Validation set: {X_val.shape}, period: {val.index.min()} to {val.index.max()}")
print(f"\nNumber of features: {len(feature_cols)}")

## 6. Baseline Models

In [ ]:
# Baseline 1: Naive forecast (today's price as tomorrow's forecast)
y_val_naive = val['GOLD_PRICE']
y_val_naive = y_val_naive.reindex(y_val.index)

rmse_naive = np.sqrt(mean_squared_error(y_val, y_val_naive))
mae_naive = mean_absolute_error(y_val, y_val_naive)

print(f"Naive Baseline (today's price):")
print(f"  RMSE: {rmse_naive:.2f}")
print(f"  MAE: {mae_naive:.2f}")

In [ ]:
# Baseline 2: 5-day moving average
df_model['GOLD_MA_5'] = df_model['GOLD_PRICE'].rolling(5).mean()
val_ma = df_model.loc[y_val.index, 'GOLD_MA_5']

rmse_ma = np.sqrt(mean_squared_error(y_val, val_ma))
mae_ma = mean_absolute_error(y_val, val_ma)

print(f"\n5-Day Moving Average Baseline:")
print(f"  RMSE: {rmse_ma:.2f}")
print(f"  MAE: {mae_ma:.2f}")

## 7. Train XGBoost Model

In [ ]:
# Train XGBoost model
xgb_model = XGBRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.9,
    colsample_bytree=0.9,
    random_state=42,
    n_jobs=-1
)

print("Training XGBoost model...")
xgb_model.fit(X_train, y_train)
print("Model training complete!")

In [ ]:
# Make predictions
y_val_pred = xgb_model.predict(X_val)

# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
mae = mean_absolute_error(y_val, y_val_pred)
mape = np.mean(np.abs((y_val - y_val_pred) / y_val)) * 100
r2 = r2_score(y_val, y_val_pred)

print(f"\nXGBoost Model Performance:")
print(f"  RMSE: {rmse:.2f}")
print(f"  MAE: {mae:.2f}")
print(f"  MAPE: {mape:.2f}%")
print(f"  R²: {r2:.4f}")

print(f"\nImprovement over baselines:")
print(f"  vs Naive: {((rmse_naive - rmse) / rmse_naive * 100):.1f}% reduction in RMSE")
print(f"  vs 5-day MA: {((rmse_ma - rmse) / rmse_ma * 100):.1f}% reduction in RMSE")

## 8. Visualizations

In [ ]:
# Plot actual vs predicted
plt.figure(figsize=(14, 5))
plt.plot(y_val.index, y_val, label="Actual", alpha=0.8, linewidth=1.5)
plt.plot(y_val.index, y_val_pred, label="Predicted (XGBoost)", alpha=0.8, linewidth=1.5)
plt.title(f"Gold Price – Actual vs Predicted (RMSE={rmse:.2f}, R²={r2:.4f})", fontsize=14)
plt.xlabel('Date')
plt.ylabel('Gold Price (USD)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(OUTPUTS_PLOTS_DIR / "gold_actual_vs_predicted_xgb.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Feature importance
importances = xgb_model.feature_importances_
fi = pd.Series(importances, index=feature_cols).sort_values(ascending=False).head(20)

plt.figure(figsize=(10, 8))
fi.sort_values().plot(kind="barh", color='steelblue')
plt.title("Top 20 Feature Importances (XGBoost)", fontsize=14)
plt.xlabel('Importance')
plt.tight_layout()
plt.savefig(OUTPUTS_PLOTS_DIR / "feature_importance_xgb.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Residual plot
residuals = y_val - y_val_pred

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.scatter(y_val_pred, residuals, alpha=0.5)
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('Predicted Values')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(residuals, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Residuals')
plt.ylabel('Frequency')
plt.title('Residual Distribution')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUTS_PLOTS_DIR / "residual_analysis.png", dpi=300, bbox_inches='tight')
plt.show()

## 9. Save Model and Forecasts

In [ ]:
# Save model
model_path = MODELS_DIR / "gold_xgb_model.pkl"
with open(model_path, "wb") as f:
    pickle.dump(xgb_model, f)

print(f"Model saved to: {model_path}")

In [ ]:
# Save forecasts
forecast_df = pd.DataFrame({
    "DATE": y_val.index,
    "GOLD_ACTUAL": y_val.values,
    "GOLD_PREDICTED": y_val_pred
})

forecast_path = OUTPUTS_FORECASTS_DIR / "gold_val_forecasts_xgb.csv"
forecast_df.to_csv(forecast_path, index=False)

print(f"Forecasts saved to: {forecast_path}")
print(f"\nFirst few rows:")
forecast_df.head()

In [ ]:
# Save feature columns for later use
feature_info = {
    'feature_cols': feature_cols,
    'metrics': {
        'rmse': rmse,
        'mae': mae,
        'mape': mape,
        'r2': r2
    }
}

feature_path = MODELS_DIR / "feature_info.pkl"
with open(feature_path, "wb") as f:
    pickle.dump(feature_info, f)

print(f"Feature info saved to: {feature_path}")

## Summary

This notebook has:
1. Loaded and cleaned gold, silver, and geopolitical risk data
2. Created engineered features (lags, rolling stats, calendar features)
3. Built baseline models for comparison
4. Trained an XGBoost model to predict next-day gold prices
5. Evaluated model performance with multiple metrics
6. Visualized results and feature importance
7. Saved the trained model and forecasts

Next steps:
- Run the Streamlit app for interactive exploration
- Consider hyperparameter tuning
- Explore SHAP values for model interpretability
- Test on different time periods